In [1]:
import numpy as np
import seaborn as sns
import pandas as pd

health = pd.read_csv("SYB64_325_202110_Expenditure on health.csv", encoding='latin-1')
health.head()

def recent(df):
    new_df = ((df.sort_values(by=['Year']).drop_duplicates(subset=['Area Name'], keep='last')).sort_values(by = ['Area Name']))
    new_df = new_df.drop(columns = ['Year'])
    return new_df

def clean(dataset):
    codes = dataset.groupby("Area Code").count().index
    i = 0
    data1 = [pd.DataFrame()] * len(codes)
    curName = ""
    for ind in codes:
        data1[i] = dataset[dataset["Area Code"] == ind]
        curName = data1[i]["Area Name"].reset_index(drop=True)[0]
        data1[i] = data1[i].pivot_table("Value",index="Year",columns="Series",aggfunc='mean')
        data1[i]["Area Code"] = [ind] * len(data1[i])
        data1[i]["Area Name"] = [curName] * len(data1[i])
        data1[i]=data1[i].reset_index()
        i = i+1

    data_final = data1[0]
    for ind in range(len(data1)):
        if(ind!=0):
            data_final = pd.concat([data_final, data1[ind]])
    data_final = data_final.reset_index(drop=True)
    return data_final

data_health = pd.read_csv("SYB64_325_202110_Expenditure on health.csv", encoding='latin-1')
data_growth = pd.read_csv("SYB61_253_Population Growth Rates in Urban areas and Capital cities.csv", encoding = "latin-1")
data_pop = pd.read_csv("SYB64_1_202110_Population, Surface Area and Density.csv", encoding = "latin-1")
data_trade = pd.read_csv("SYB64_123_202110_Total Imports Exports and Balance of Trade.csv", encoding = "latin-1")
data_cpi = pd.read_csv("SYB64_128_202110_Consumer Price Index.csv", encoding = "latin-1")
data_land = pd.read_csv("SYB64_145_202110_Land.csv", encoding = "latin-1")
data_employment = pd.read_csv("SYB64_200_202110_Employment.csv", encoding = "latin-1")
data_gdp = pd.read_csv("SYB64_230_202110_GDP and GDP Per Capita.csv", encoding = "latin-1")
data_mortality = pd.read_csv("SYB64_246_202110_Population Growth, Fertility and Mortality Indicators.csv", encoding = "latin-1")
data_health = pd.read_csv("SYB64_325_202110_Expenditure on health.csv", encoding = "latin-1")
data_death = pd.read_csv("Death Cause Reason by Country.csv", encoding = "latin-1")

health = clean(data_health)
growth = clean(data_growth)
pop = clean(data_pop)
trade = clean(data_trade)
cpi = clean(data_cpi)
land = clean(data_land)
employment = clean(data_employment)
gdp = clean(data_gdp)
mortality = clean(data_mortality)

rec_health = recent(health)
rec_growth = recent(growth)
rec_pop = recent(pop)
rec_trade = recent(trade)
rec_cpi = recent(cpi)
rec_land = recent(land)
rec_employment = recent(employment)
rec_gdp = recent(gdp)
rec_mortality = recent(mortality)

df_merge = [rec_growth, rec_pop, rec_trade, rec_cpi, rec_land, rec_employment, rec_gdp, rec_mortality]
merged = rec_health
for df in df_merge:
    merged = merged.merge(df, how="left", on=['Area Name', 'Area Code'])

merged.head()

combined = pd.merge(merged, data_death, how = "inner", left_on = "Area Name", right_on = "Country Name")
#cpi.head()
#data_death.head()
#combined.head()

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

variables = combined[["Current health expenditure (% of GDP)", "Covid-19 Deaths"]].dropna()

dataset = variables[["Current health expenditure (% of GDP)"]].values
target = variables["Covid-19 Deaths"].values

lin_reg = LinearRegression()
lin_reg.fit(X = dataset, y = target)
variables["Predicted deaths"] = lin_reg.predict(dataset)

mse = mean_squared_error(variables["Covid-19 Deaths"].values, variables["Predicted deaths"].values) 
r2 = r2_score(variables["Covid-19 Deaths"].values, variables["Predicted deaths"].values) 

print(mse, r2)

627817861.6843169 0.016486744884904092


In [13]:
variables2 = combined[["Urban population (percent)", "Interpersonal violence"]].dropna()

dataset = variables2[["Urban population (percent)"]].values
target = variables2["Interpersonal violence"].values

lin_reg = LinearRegression()
lin_reg.fit(X = dataset, y = target)
variables2["Predicted deaths"] = lin_reg.predict(dataset)

mse = mean_squared_error(variables2["Interpersonal violence"].values, variables2["Predicted deaths"].values) 
r2 = r2_score(variables2["Interpersonal violence"].values, variables2["Predicted deaths"].values) 

print(mse, r2)

49579.89025452992 0.1366228864424217


In [14]:
variables3 = combined[["Population annual rate of increase (percent)", "Interpersonal violence"]].dropna()

dataset = variables3[["Population annual rate of increase (percent)"]].values
target = variables3["Interpersonal violence"].values

lin_reg = LinearRegression()
lin_reg.fit(X = dataset, y = target)
variables3["Predicted deaths"] = lin_reg.predict(dataset)

mse = mean_squared_error(variables3["Interpersonal violence"].values, variables3["Predicted deaths"].values) 
r2 = r2_score(variables3["Interpersonal violence"].values, variables3["Predicted deaths"].values) 

print(mse, r2)

50300828.18900577 0.0006456897345733514


In [15]:
variables4 = combined[["Life expectancy at birth for both sexes (years)", "Covid-19 Deaths"]].dropna()

dataset = variables4[["Life expectancy at birth for both sexes (years)"]].values
target = variables4["Covid-19 Deaths"].values

lin_reg = LinearRegression()
lin_reg.fit(X = dataset, y = target)
variables4["Predicted deaths"] = lin_reg.predict(dataset)

mse = mean_squared_error(variables4["Covid-19 Deaths"].values, variables4["Predicted deaths"].values) 
r2 = r2_score(variables4["Covid-19 Deaths"].values, variables4["Predicted deaths"].values) 

print(mse, r2)

665704642.0780625 0.026309100838424415


In [16]:
variables5 = combined[["Consumer price index: General", "Self-harm"]].dropna()

dataset = variables5[["Consumer price index: General"]].values
target = variables5["Self-harm"].values

lin_reg = LinearRegression()
lin_reg.fit(X = dataset, y = target)
variables5["Predicted deaths"] = lin_reg.predict(dataset)

mse = mean_squared_error(variables5["Self-harm"].values, variables5["Predicted deaths"].values) 
r2 = r2_score(variables5["Self-harm"].values, variables5["Predicted deaths"].values) 

print(mse, r2)

343919454.0355641 0.002176694497578402
